In [223]:
import numpy as np
import pandas as pd
from pathlib import Path
from math import radians, cos, sin, asin, sqrt
import datetime as dt
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold,  KFold,train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler


pd.set_option("display.max_columns", 100)

#mount google drive folder
from google.colab import drive
drive.mount('/content/gdrive')
data_path = "/content/gdrive/My Drive/FareClassification/"

#read csv files
train_df = pd.read_csv(data_path + "train.csv", index_col="tripid" )
test_df = pd.read_csv(data_path + "test.csv",  index_col="tripid" )

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


1.Preprocessing & Feature Engineering




In [224]:
dataset_map = {"incorrect": 0, "correct":1} 
train_df['label'] = train_df["label"].map(dataset_map) #convert "label" column to binary values
train_df.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label
tripid,,,,,,,,,,,,,
189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,1
189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,1
189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,1
189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,1
189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,1


In [225]:
train_df.isnull().sum() #check for missing values

additional_fare              202
duration                     202
meter_waiting                202
meter_waiting_fare           202
meter_waiting_till_pickup    202
pickup_time                    0
drop_time                      0
pick_lat                       0
pick_lon                       0
drop_lat                       0
drop_lon                       0
fare                         137
label                          0
dtype: int64

In [226]:
#fill missing values of "duration" column using "drop_time" and "pickup time"
train_df['calculated_duration'] = (pd.to_datetime(train_df['drop_time']) - pd.to_datetime(train_df['pickup_time']))/np.timedelta64(1,'s')
train_df['duration'] = train_df['duration'].fillna(train_df ['calculated_duration'])
train_df = train_df.drop(train_df[['calculated_duration','pickup_time','drop_time']], axis=1) #drop "drop_time" and "pickup time" column
train_df.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,fare,label
tripid,,,,,,,,,,,
189123628,10.5,834.0,56.0,0.0000,64.0,6.86252,79.8993,6.90330,79.8783,270.32,1
189125358,10.5,791.0,47.0,0.0000,134.0,6.88589,79.8984,6.91373,79.8923,197.85,1
189125719,10.5,1087.0,80.0,0.0000,61.0,6.90839,79.8651,6.93669,79.9146,301.64,1
189127273,10.5,598.0,271.0,15.6638,68.0,6.92570,79.8895,6.92748,79.8971,82.30,1
189128020,NaN,1020.0,NaN,NaN,NaN,6.87441,79.8615,6.84478,79.9290,358.39,1


In [227]:
train_df.isnull().sum()

additional_fare              202
duration                       0
meter_waiting                202
meter_waiting_fare           202
meter_waiting_till_pickup    202
pick_lat                       0
pick_lon                       0
drop_lat                       0
drop_lon                       0
fare                         137
label                          0
dtype: int64

In [228]:
train_df = train_df.fillna(train_df.mean()) #fill remaining missing values by usin corresponding mean

In [229]:
train_df.isnull().sum()

additional_fare              0
duration                     0
meter_waiting                0
meter_waiting_fare           0
meter_waiting_till_pickup    0
pick_lat                     0
pick_lon                     0
drop_lat                     0
drop_lon                     0
fare                         0
label                        0
dtype: int64

In [230]:
def calc_haversine(lon1, lat1, lon2, lat2):
    #convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2]) 

    # haversine formula
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    r = 6371
    haversine_distance = c * r
    return haversine_distance

#create new column "distance" using havesine distance
train_df['distance'] = calc_haversine(train_df['pick_lat'], train_df['pick_lon'], train_df['drop_lat'] , train_df['drop_lon'])
train_df = train_df.drop(train_df[['pick_lat','pick_lon','drop_lat','drop_lon']], axis=1) #drop columns
train_df.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,fare,label,distance
tripid,,,,,,,,
189123628,10.500000,834.0,56.000000,0.000000,64.000000,270.32,1,2.467063
189125358,10.500000,791.0,47.000000,0.000000,134.000000,197.85,1,0.868942
189125719,10.500000,1087.0,80.000000,0.000000,61.000000,301.64,1,5.531798
189127273,10.500000,598.0,271.000000,15.663800,68.000000,82.30,1,0.845795
189128020,13.719651,1020.0,629.074231,32.057666,112.466832,358.39,1,7.527884


In [231]:
train_df = train_df.drop(train_df[['meter_waiting_till_pickup']], axis=1)

In [232]:
X_train = train_df.drop(['label'],axis='columns')
Y_train = train_df['label']
Y_train.head()

tripid
189123628    1
189125358    1
189125719    1
189127273    1
189128020    1
Name: label, dtype: int64

In [233]:
train_df.head(3)

,additional_fare,duration,meter_waiting,meter_waiting_fare,fare,label,distance
tripid,,,,,,,
189123628,10.5,834.0,56.0,0.0,270.32,1,2.467063
189125358,10.5,791.0,47.0,0.0,197.85,1,0.868942
189125719,10.5,1087.0,80.0,0.0,301.64,1,5.531798


2.Model creation & Training

In [239]:
random_grid = {'n_estimators': range(100,500,100),
               'max_depth': range(5,20,1),
               'min_samples_leaf':range(2,5,1),
              'max_features':['auto','sqrt','log2'],
              'bootstrap': [True, False],
              'min_samples_split': range(2,5,1)}

rforest = RandomForestClassifier()
xgb_= XGBClassifier()

rfcv = RandomizedSearchCV(rforest, random_grid,cv=5)
xgbcv = RandomizedSearchCV(xgb_, random_grid,cv=5)

rfcv.fit(X_train, Y_train)
xgbcv.fit(X_train, Y_train)

# Print the tuned parameters and score
print("Tuned Parameters for random forest: {}".format(rfcv.best_params_))
print("Best score for random forest is {}".format(rfcv.best_score_))
print("Tuned Parameters for xgboost: {}".format(xgbcv.best_params_))
print("Best score for xgboost is {}".format(xgbcv.best_score_))


Tuned Parameters for random forest: {'n_estimators': 100, 'min_samples_split': 3, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 18, 'bootstrap': False}
Best score for random forest is 0.9441659253083626
Tuned Parameters for xgboost: {'n_estimators': 200, 'min_samples_split': 4, 'min_samples_leaf': 4, 'max_features': 'auto', 'max_depth': 13, 'bootstrap': False}
Best score for xgboost is 0.9450975119168052


In [240]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])

numeric_features = train_df.select_dtypes(include=['int64', 'float64']).drop(['label'], axis=1).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)])


In [241]:
#test with different classifiers
knn = KNeighborsClassifier(n_neighbors=11, p=2,metric='euclidean')
rft = RandomForestClassifier(n_jobs=2,random_state=0)
LR = LogisticRegression(random_state = 2)
xgb= XGBClassifier( learning_rate =0.14, n_estimators=570)

In [242]:
kfold = StratifiedKFold(n_splits=10)

In [243]:
c_results = []
c_results.append(cross_val_score(xgb, X_train, y = Y_train, scoring = 'f1', cv = kfold, n_jobs=4)) 

c_means = 0
c_std = 0
for c_result in c_results:
    c_means = c_result.mean()
    c_std  = c_result.std()

c_means

0.9699174488429898

In [244]:
pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', xgb)])

In [245]:
x_train, x_eval, y_eval, y_eval = train_test_split(X_train, Y_train, test_size=0.25,stratify=Y_train)

In [246]:
pipe.fit(x_eval, y_eval)
y_pred = pipe.predict(x_eval)

In [247]:
f1_score(y_eval, y_pred,average='weighted')

0.9838271960087079

In [248]:
pipe.fit(X_train,Y_train)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='mean',
                                                               

3.Test Data Preprocessing

In [249]:
test_df.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare
tripid,,,,,,,,,,,,
213284604,10.5,924,42,2.4486,148,2/1/2020 0:38,2/1/2020 0:53,6.83454,79.8750,6.77490,79.8840,289.27
213286352,10.5,4249,20,0.0000,91,2/1/2020 1:02,2/1/2020 2:13,6.91168,79.8723,6.55091,79.9706,1912.70
213293973,10.5,1552,255,2.6588,23,2/1/2020 5:02,2/1/2020 5:28,6.92145,79.8478,6.90539,79.8989,394.00
213294622,10.5,462,16,0.0000,198,2/1/2020 5:30,2/1/2020 5:38,6.77433,79.9416,6.80401,79.9407,154.32
213298687,10.5,814,392,12.3692,69,2/1/2020 7:00,2/1/2020 7:14,6.97968,79.9130,6.98875,79.8914,147.47


In [250]:
test_df['distance'] = calc_haversine(test_df['pick_lat'], test_df['pick_lon'], test_df['drop_lat'] , test_df['drop_lon'])

In [251]:
test_df = test_df.drop(['pick_lat','pick_lon','drop_lat','drop_lon'],axis='columns')
test_df = test_df.drop(['pickup_time','drop_time','meter_waiting_till_pickup'],axis='columns')

In [252]:
test_df.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,fare,distance
tripid,,,,,,
213284604,10.5,924,42,2.4486,289.27,1.536052
213286352,10.5,4249,20,0.0000,1912.70,12.990640
213293973,10.5,1552,255,2.6588,394.00,5.690729
213294622,10.5,462,16,0.0000,154.32,0.585046
213298687,10.5,814,392,12.3692,147.47,2.408311


In [253]:
test_df.shape

(8576, 6)

4.Predicting Results



In [254]:
result = pipe.predict(test_df)
test_df['prediction'] = result

In [255]:
submission_df = test_df.drop(['additional_fare','distance','duration','fare','meter_waiting','meter_waiting_fare'],axis='columns')

In [256]:
submission_df.head()

,prediction
tripid,
213284604,1
213286352,0
213293973,1
213294622,1
213298687,1


5.Write Generated csv file  to the drive

In [257]:
submission_df.to_csv(data_path +'submission.csv', index=True)